In [1]:
import pandas as pd

In [3]:
data = pd.read_csv('us-layoff-analysis.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2303 entries, 0 to 2302
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   state                   2303 non-null   object 
 1   company                 2303 non-null   object 
 2   city                    2209 non-null   object 
 3   number_of_workers       2292 non-null   float64
 4   warn_received_date      2303 non-null   object 
 5   effective_date          1956 non-null   object 
 6   action                  1865 non-null   object 
 7   temporary_or_permanent  580 non-null    object 
 8   union                   148 non-null    object 
 9   county                  1476 non-null   object 
 10  industry                1324 non-null   object 
 11  notes                   457 non-null    object 
 12  source                  2303 non-null   object 
dtypes: float64(1), object(12)
memory usage: 234.0+ KB


In [8]:
data_cleaned = data.dropna(subset=['industry'])
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1324 entries, 9 to 2300
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   state                   1324 non-null   object 
 1   company                 1324 non-null   object 
 2   city                    1264 non-null   object 
 3   number_of_workers       1322 non-null   float64
 4   warn_received_date      1324 non-null   object 
 5   effective_date          1222 non-null   object 
 6   action                  1310 non-null   object 
 7   temporary_or_permanent  297 non-null    object 
 8   union                   101 non-null    object 
 9   county                  949 non-null    object 
 10  industry                1324 non-null   object 
 11  notes                   124 non-null    object 
 12  source                  1324 non-null   object 
dtypes: float64(1), object(12)
memory usage: 144.8+ KB


In [9]:
data_cleaned

,state,company,city,number_of_workers,warn_received_date,effective_date,action,temporary_or_permanent,union,county,industry,notes,source
9,Wisconsin,Milwaukee Forge LLC Update,Milwaukee,60.0,2025-08-19,2025-10-30,Change to Layoff Schedule; Change to Number of...,NaN,"USW Local No. 3205, IAMAW Lodge D.S. 140",Milwaukee,Iron & Steel Forging,Update to 6/20/2025 notice,WARN
10,Kentucky,AHF Products Manufacturing,NaN,134.0,2025-08-19,2025-12-11,Closure,NaN,NaN,Pulaski,321920,NaN,WARN
11,Maryland,"Catalent Maryland, Inc.",Hanover,5.0,2025-08-19,2025-10-17,Mass Layoff - No Recall,NaN,NaN,NaN,325412,"7031 Ridge Road Hanover, MD 21076",WARN
12,California,Town Hospice & Palliative Care LLC dba Suncres...,6465 Sycamore Canyon Blvd. Suie 150 Riverside...,23.0,2025-08-19,2025-11-20,Closure Permanent,NaN,NaN,Riverside County,62 Healthcare and Social Assistance,NaN,WARN
13,California,Northrop Grumman,6379 San Ignacio Avenue San Jose CA 95119,78.0,2025-08-19,2025-10-30,Closure Permanent,NaN,NaN,Santa Clara County,54 Professional Scientific and Technical Services,NaN,WARN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2296,Minnesota,The Clover,Rosemount,1.0,2024-12-31,2025-01-30,Closing,Small Layoff Services,NaN,NaN,Dining Establishments,NaN,WARN
2297,Minnesota,Irish on Grand,St. Paul,1.0,2024-12-31,2025-03-29,Closing,Small Layoff Services,NaN,NaN,Retail,NaN,WARN
2298,Minnesota,Petek Trading Co.,St. Paul,1.0,2024-12-31,2025-02-15,Closing,Small Layoff Services,NaN,NaN,Retail,NaN,WARN
2299,Minnesota,BANK Restaurant,Minneapolis,1.0,2024-12-31,2025-01-28,Closing,Small Layoff Services,NaN,NaN,Accommodation,NaN,WARN


In [12]:
from rapidfuzz import process

# Standardized industry categories
standard_categories = [
    "Manufacturing",
    "Retail Trade",
    "Healthcare",
    "Accommodation & Food Services",
    "Professional Services",
    "Other Services",
    "Construction",
    "Transportation & Warehousing",
    "Information",
    "Finance & Insurance",
    "Education Services",
    "Public Administration",
    "Agriculture, Forestry, Fishing & Hunting",
    "Mining, Quarrying, & Oil/Gas Extraction",
    "Utilities",
    "Arts, Entertainment & Recreation"
]

# Function to normalize industries
def normalize_industry_fuzzy(value):
    if pd.isna(value):
        return "Other"
    
    value_str = str(value).strip()
    
    # 1. Try numeric NAICS code first
    try:
        code = int(value_str)
        sector_code = str(code)[:2]
        naics_sector_mapping = {
            '11': 'Agriculture, Forestry, Fishing & Hunting',
            '21': 'Mining, Quarrying, & Oil/Gas Extraction',
            '22': 'Utilities',
            '23': 'Construction',
            '31': 'Manufacturing',
            '32': 'Manufacturing',
            '33': 'Manufacturing',
            '42': 'Wholesale Trade',
            '44': 'Retail Trade',
            '45': 'Retail Trade',
            '48': 'Transportation & Warehousing',
            '49': 'Transportation & Warehousing',
            '51': 'Information',
            '52': 'Finance & Insurance',
            '53': 'Real Estate & Rental & Leasing',
            '54': 'Professional Services',
            '55': 'Management of Companies',
            '56': 'Administrative & Support Services',
            '61': 'Education Services',
            '62': 'Healthcare',
            '71': 'Arts, Entertainment & Recreation',
            '72': 'Accommodation & Food Services',
            '81': 'Other Services',
            '92': 'Public Administration'
        }
        return naics_sector_mapping.get(sector_code, "Other")
    except:
        pass
    
    # 2. Fuzzy match free-text to standard categories
    match, score, _ = process.extractOne(value_str, standard_categories)
    # Threshold to avoid wrong matches
    if score >= 70:
        return match
    else:
        return "Other"

# Apply normalization
data_cleaned['industry_normalized'] = data_cleaned['industry'].apply(normalize_industry_fuzzy)

C:\Users\ryuji\AppData\Local\Temp\ipykernel_18916\3441909251.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['industry_normalized'] = data_cleaned['industry'].apply(normalize_industry_fuzzy)


KeyError: 'Product'

In [13]:
print(data_cleaned['industry_normalized'].value_counts())

industry_normalized
Manufacturing                               222
Accommodation & Food Services               197
Healthcare                                  188
Other                                       161
Retail Trade                                149
Transportation & Warehousing                 95
Professional Services                        59
Finance & Insurance                          52
Information                                  48
Education Services                           47
Other Services                               44
Arts, Entertainment & Recreation             18
Administrative & Support Services            10
Agriculture, Forestry, Fishing & Hunting      9
Mining, Quarrying, & Oil/Gas Extraction       7
Construction                                  6
Utilities                                     5
Public Administration                         4
Wholesale Trade                               3
Name: count, dtype: int64


In [17]:
layoff_data = data_cleaned[data_cleaned['action'].str.contains('Layoff', na=False)]
layoff_data.head()

,state,company,city,number_of_workers,warn_received_date,effective_date,action,temporary_or_permanent,union,county,industry,notes,source,industry_normalized
9,Wisconsin,Milwaukee Forge LLC Update,Milwaukee,60.0,2025-08-19,2025-10-30,Change to Layoff Schedule; Change to Number of...,NaN,"USW Local No. 3205, IAMAW Lodge D.S. 140",Milwaukee,Iron & Steel Forging,Update to 6/20/2025 notice,WARN,"Agriculture, Forestry, Fishing & Hunting"
11,Maryland,"Catalent Maryland, Inc.",Hanover,5.0,2025-08-19,2025-10-17,Mass Layoff - No Recall,NaN,NaN,NaN,325412,"7031 Ridge Road Hanover, MD 21076",WARN,Manufacturing
18,California,Corteva Agriscience LLC,901 Loveridge Road Pittsburg CA 94565,10.0,2025-08-18,2025-10-23,Layoff Permanent,NaN,NaN,Contra Costa County,54 Professional Scientific and Technical Services,NaN,WARN,Accommodation & Food Services
19,California,Kaiser Foundation Hospitals,One Kaiser Plaza Oakland CA 94612,2.0,2025-08-18,2025-08-03,Layoff Permanent,NaN,NaN,Alameda County,62 Healthcare and Social Assistance,NaN,WARN,Healthcare
20,California,Kaiser Foundation Hospitals (1830),1830 California Ave Corona CA 92881,2.0,2025-08-18,2025-08-03,Layoff Permanent,NaN,NaN,Riverside County,62 Healthcare and Social Assistance,NaN,WARN,Healthcare


In [23]:
layoff_data.to_csv('layoff_data.csv')

In [24]:
layoff_data.sort_values(by='effective_date')

,state,company,city,number_of_workers,warn_received_date,effective_date,action,temporary_or_permanent,union,county,industry,notes,source,industry_normalized
1908,California,"CareFusion Resources, LLC, a subsidiary of Bec...",3750 Torrey View Court Building A & B San Dieg...,48.0,2025-02-11,2024-12-19,Layoff Permanent,NaN,NaN,San Diego County,42 Wholesale Trade,NaN,WARN,Retail Trade
2192,Illinois,The Computing Technology Industry Association,Downers Grove,47.0,2025-01-13,2025-01-08,Layoff,Permanent,No,DuPage,Technology Traninig & Certification,NaN,WARN,Finance & Insurance
2174,California,Aurora Solar Inc.,"153 Kearny St., 5th Floor San Francisco CA 94108",58.0,2025-01-14,2025-01-09,Layoff Permanent,NaN,NaN,San Francisco County,51 Information,NaN,WARN,Information
2208,Iowa,"IES Communications, LLC",Altoona,28.0,2025-01-09,2025-01-09,Mass Layoff,NaN,NaN,Polk,Construction,NaN,WARN,Construction
2194,California,ACLU of Southern California,"1313 W 8th Street, #200 Los Angeles CA 90017",8.0,2025-01-13,2025-01-09,Layoff Permanent,NaN,NaN,Los Angeles County,81 Other Services Except Public Administration,NaN,WARN,Other Services
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2123,Missouri,Eaton Bussman,Ellisville,57.0,2025-01-26,NaN,Layoff,NaN,NaN,St. Louis,Manufacturing,NaN,WARN,Manufacturing
2125,Florida,Legoland America,Winter Haven,234.0,2025-01-24,NaN,Layoff,Permanent,NaN,NaN,"Arts, Entertainment, and Recreation",NaN,WARN,"Arts, Entertainment & Recreation"
2210,Florida,"Stryker Employment Company, LLC",Lakeland,11.0,2025-01-09,NaN,Layoff,Permanent,NaN,NaN,Manufacturing,part of layoff event affecting 500 workers,WARN,Manufacturing
2243,Illinois,"Innovation Group North America, Inc.",Schaumburg,105.0,2025-01-06,NaN,Layoff,Permanent,No,Cook,Finance and Insurance,NaN,WARN,Finance & Insurance
